# PHE Download

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

In [78]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [79]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

In [80]:
from requests import get
from json import dumps

class Area(Printable):
    def __init__(self, areaType, areaName):
        """Initialisise the partial results"""

        self.areaType = areaType
        self.areaName = areaName
        self.csvName = areaName.lower().replace(' ', '_') + '.csv'
        
        for word in 'of', 'and', 'the':
            self.csvName = self.csvName.replace(word + '_', '')


    def download(self, dirName, demographics):
        """Download cases by specimen date age demographics"""

        try:
            ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"

            filters = [
                f"areaType={self.areaType}",
                f"areaName={self.areaName}"
            ]

            structure = {
                "date": "date",
                "areaName": "areaName",
                demographics: demographics,
            }

            api_params = {
                "filters": str.join(";", filters),
                "structure": dumps(structure, separators=(",", ":")),
                "format": "csv"
            }

            response = get(ENDPOINT, params=api_params, timeout=10)
            assert response.status_code == 200, f"Failed request for {fmt}: {response.text}"
            
            path = os.path.join(projdir, "data", "download", "phe", dirName, self.areaType)
            if not os.path.exists(path):
                os.makedirs(path)

            fn = os.path.join(path, self.csvName)
            with open(fn, 'w') as f:
                f.write(response.content.decode())

        except:
            print(f"Failed to retrieve {dirName} for {self.areaName}")
            raise


    def downloadCases(self):
        """Download cases by specimen date age demographics"""

        self.download("cases", "newCasesBySpecimenDateAgeDemographics")
            

    def downloadDeaths(self):
        """Download deaths within 28 days of positive test by date of death age demographics"""

        self.download("deaths", "newDeaths28DaysByDeathDateAgeDemographics")

In [81]:
areaNames = ['North West', 'North East', 'Yorkshire and The Humber',
             'West Midlands', 'East Midlands',
             'East of England', 'London', 'South East', 
             'South West'
           ]

areaName = 'England'
print(f"Downloading {areaName}...")
area = Area("nation", areaName)
area.downloadCases()
area.downloadDeaths()

for areaName in areaNames:
    print(f"Downloading {areaName}...")
    area = Area("region", areaName)
    area.downloadCases()
    area.downloadDeaths()

print("All done!")

Processing England...
Processing North West...
Processing North East...
Processing Yorkshire and The Humber...
Processing West Midlands...
Processing East Midlands...
Processing East of England...
Processing London...
Processing South East...
Processing South West...
All done!
